In [110]:
import os
import pandas as pd 
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from scipy.stats import t as tdist 
import scipy.stats as stats
from statsmodels.stats.outliers_influence import summary_table
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn import datasets, linear_model

In [111]:
os.getcwd()

'/Users/sahiljain/Downloads'

In [112]:
os.chdir("/Users/sahiljain/Downloads/")

In [113]:
hitters = pd.read_csv('hitters.csv')

(A) Calculate the VIF for each of the explanatory variables. Comment 
on weather multicollinearity appears to be an isse. If it is,
identify the three explanatory variables that are most seriously 
affected by the issue. 

First we will do a MLR of the data, then calculate the VIF. 

In [114]:
y, X = dmatrices('Salary ~ AtBat + Hits + HmRun +  Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists + Errors + NewLeague', data=hitters, return_type="dataframe")
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
result = sm.OLS(y, X).fit()

In [115]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     15.39
Date:                Tue, 21 Nov 2017   Prob (F-statistic):           7.84e-32
Time:                        18:55:42   Log-Likelihood:                -1876.2
No. Observations:                 263   AIC:                             3792.
Df Residuals:                     243   BIC:                             3864.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        163.1036     90.779      1.797      0.074     -15.710     341.917
League[T.N]       62.5994     79.261      0.790      0.430     -93.528     218.727
Division[T.W]   -116.8492     40.367     -2.895      0.004    -196.363     -37.335
NewLeague[T.N]   -24.7623     79.003     -0.313      0.754    -180.380     130.855
AtBat             -1.9799      0.634     -3.123      0.002      -3.229      -0.731
Hits               7.5008      2.378      3.155      0.002       2.818      12.184
HmRun              4.3309      6.201      0.698      0.486      -7.885      16.546
Runs              -2.3762      2.981     -0.797      0.426      -8.248       3.495
RBI               -1.0450      2.601     -0.402      0.688      -6.168       4.078
Walks              6.2313      1.829      3.408      0.001       2.630       9.833
Years             -3.4891     12.412     -0.281      0.779     -27.938      20.960
CAtBat            -0.1713      0.135     -1.267      0.206      -0.438       0.095
CHits              0.1340      0.675      0.199      0.843      -1.195       1.463
CHmRun            -0.1729      1.617     -0.107      0.915      -3.358       3.013
CRuns              1.4543      0.750      1.938      0.054      -0.024       2.933
CRBI               0.8077      0.693      1.166      0.245      -0.557       2.172
CWalks            -0.8116      0.328     -2.474      0.014      -1.458      -0.165
PutOuts            0.2819      0.077      3.640      0.000       0.129       0.434
Assists            0.3711      0.221      1.678      0.095      -0.065       0.807
Errors            -3.3608      4.392     -0.765      0.445     -12.011       5.290
==============================================================================
Omnibus:                       87.414   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              452.923
Skew:                           1.236   Prob(JB):                     4.46e-99
Kurtosis:                       8.934   Cond. No.                     2.09e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [116]:
print vif

[21.762082248706328, 4.1341152908350125, 1.0753975007072221, 4.0990634495651141, 22.944365916840002, 30.281255330499725, 7.7586678548431989, 15.246417502100771, 11.9217150823485, 4.1487119716733201, 9.3132799023356263, 251.5611595648281, 502.95428903838939, 46.488461529704331, 162.52081015034895, 131.96585767572046, 19.744105013833646, 1.2363169622521883, 2.7093409367595696, 2.2145434666868042]


Yes, multicolinearity does appear to be an issue here as some of the 
variables are seriously affected. Most seriously affected variables 
are :
(1) CAtBat 
(2) CHits 
(3) CRuns
If we take a closer look, Python also calculates the VIF for categorical variables as well as for the intercept so the first four outputs in the vif are of categorical and response variable.

(F) In this part you will compare the predictive performance of four 
models:

(i) The full model with all 19 explanatory variables.
(ii) The optimal model identified in part (b).
(iii) The best model from parts (c)-(e) (i.e., the best stepwise-selection model).
(iv) The model that is considered optimal with respect to the Bayesian
Information Criterion (BIC) which contains the variables AtBat, Hits, 
Walks, CRBI, Division and PutOuts.

Randomly split the observed data into a training set (containing 
roughly 80% of all of the data) and a held-out test set (containing 
roughly 20% of all of the data).Calculate the predictive root-mean-
square error (RMSE) for each of the four models. Which model appears 
to be most appropriate? Justify why this model is most appropriate.

First we will split the data into training and test set 

In [117]:
X = hitters.drop("Salary", 1)
y = hitters["Salary"]

In [118]:
# Training and Test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)

(I) The full model with all 19 explanatory variables. 

In [119]:
m1 = smf.ols(formula = 'Salary ~ AtBat + Hits + HmRun +  Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists + Errors + NewLeague', data=train).fit()
pred1 = m1.predict(X_test)
RMSE1 = np.sqrt(np.mean((y_test - pred1)**2))
RMSE1

302.05370688561362

(ii) The optimal model identified in part b 

In [120]:
m2 = smf.ols(formula = 'Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists', data = train).fit()
pred2 = m2.predict(X_test)
RMSE2 = np.sqrt(np.mean((y_test - pred2)**2))
RMSE2

294.47897359077996

(iii) The best stepwise selection model

In [121]:
m3 = smf.ols(formula = 'Salary ~ CRBI + Hits + PutOuts + Division + AtBat +  Walks + CWalks + CRuns + CAtBat + Assists', data = train).fit()
pred3 = m3.predict(X_test)
RMSE3 = np.sqrt(np.mean((y_test - pred3)**2))
RMSE3

296.41386222544998

(IV) Bayesian Information criterian. 

In [122]:
m4 = smf.ols('Salary ~ AtBat + Hits + Walks + CRBI + Division + PutOuts', data = train).fit()
pred4 =  m4.predict(X_test)
RMSE4 = np.sqrt(np.mean((y_test - pred4)**2))
RMSE4

281.71062614102044

From the output optimal model is the best model because of the lower RMSE, since lower RMSE is better. Also the answer and best model should change every time since training set will be different everytime. 

(G) As in part (f), you must compare the predictive performance of the same four models, but here you must determine 
the predictive accuracy (predictive RMSE) by using 10-Fold Cross Validation. Which model appears to be most 
appropriate? Justify why this model is most appropriate.

(I) All explanatory variables

In [123]:
#K-fold cross validation 
numfolds = 10
kf = KFold(n=263, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.ix[train_indices, :]; train_y = y[train_indices]
    test_X = X.ix[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m5 = smf.ols('Salary ~ AtBat + Hits + HmRun +  Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists + Errors + NewLeague', data = train).fit()   
    pred = m5.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE5 = np.sqrt(MSE/numfolds)
RMSE5

/Users/sahiljain/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


306.41804889221157

(2) The optimal model in part b

In [124]:
numfolds = 10
kf = KFold(n=263, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.ix[train_indices, :]; train_y = y[train_indices]
    test_X = X.ix[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m6 = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists', data = train).fit()   
    pred = m6.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE6 = np.sqrt(MSE/numfolds)
RMSE6

307.54704203239589

(3) The best subset model

In [125]:
#K-fold cross validation 
numfolds = 10
kf = KFold(n=263, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.ix[train_indices, :]; train_y = y[train_indices]
    test_X = X.ix[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m7 = smf.ols('Salary ~ CRBI + Hits + PutOuts + Division + AtBat +  Walks + CWalks + CRuns + CAtBat + Assists', data = train).fit()   
    pred = m7.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE7 = np.sqrt(MSE/numfolds)
RMSE7

307.24918033691949

(4) Bayesian Information Criterian 

In [126]:
numfolds = 10
kf = KFold(n=263, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.ix[train_indices, :]; train_y = y[train_indices]
    test_X = X.ix[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m8 = smf.ols('Salary ~ AtBat + Hits + Walks + CRBI + Division + PutOuts', data = train).fit()   
    pred = m8.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE8 = np.sqrt(MSE/numfolds)
RMSE8

317.70730086272999

After doing K-Folds Cross validation of 10 folds, we can determine that the best subset model (stepwise - selection) appears to be the most appropriate model, because of the root mean square error (RMSE) value since smaller the RMSE better will the predictive accuracy of the model. Same in this case the answer/better model will vary with respect to training set.  

(H) Given the estimates of predictive accuracy from parts (f) and (g) indicate which estimates you believe to be more accurate. In other words, indicate which
validation approach (i.e., cross validation vs. k-fold cross validation) you believe will most accurately estimate the predictive capability of a model. Briefly explain
your rationale.

K-fold cross validation is the much better approach as compared to cross validation because if we are proceeding with normal validation, we are reducing the dataset size by almost around 20–30%. However, In case of k-fold cross validation there won't be any such kind of reduction in dataset size moreover by doing k-fold cross validation we can prevent over fitting.

(G) Accounting for all of the analyses you’ve performed (i.e., multicollinearity, goodness-of-fit, and predictive accuracy), which model would you be most comfortable using? Briefly justify your choice. [Note: I’m not looking for a right or wrong answer here; I want to see that you can sensibly and eloquently justify your choice].

I will prefer predictive accuracy model because in predictive accuracy one can easily do different regression (step-wise) and different validations (cross, K-fold) to understand the result in a better way. Also in predictive accuracy most of the statistical insignificant variables are dropped out of the model which makes prediction even more easier. 
